In [3]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

TensorFlow 1.x selected.


In [0]:
X = tf.placeholder(tf.float32, [None, 2, 3])
seq_length = tf.placeholder(tf.int32, [None])

In [0]:
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7]],
    [[3, 4, 5], [0, 0, 0]],
    [[6, 7, 8], [6, 5, 4]],
    [[9, 0, 1], [3, 2, 1]],
])
seq_length_batch = np.array([2, 1, 2, 2])

In [12]:
lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(5, forget_bias = 1.0, state_is_tuple = True)
lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(5, forget_bias = 1.0, state_is_tuple = True)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [14]:
(output_fw, output_bw) , states = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, dtype = tf.float32,
                                                                  inputs = X, sequence_length = seq_length_batch)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
init = tf.global_variables_initializer()

In [0]:
with tf.Session() as sess:
  sess.run(init)
  fw, bw, state = sess.run([output_fw, output_bw, states], feed_dict = {X: X_batch, seq_length: seq_length_batch})

In [27]:
print(fw)
print("shape of fw : ", np.shape(fw)) # (4,2,5) 4 = 데이터 갯수, 2 = 문장 갯수, 5 = lstm셀의 노드갯수
print(bw)
print("shape of bw : ", np.shape(bw)) # bw도 fw와 같다
print(state)
print("shape of state : ", np.shape(state))

# (2,2,4,5) c=array 는 lstm 모듈에서 셀 스테이트가 되고
# h=array는 lstm 모듈에서 히든 스테이트가 된다.

# 히든 스테이트 같은경우는 fw cell 에서는 뒷부분을 갖고오고
# bw cell 에서는 앞부분을 갖고 온다

# 결론적으로 셀스테이트는 버리고 각 히든스테이트에 대해서 fw cell 히든스테이트와
# bw cell 히든스테이트를 concatenate(concat) 해준다
# 일렬로 한줄로 쭉 붙여서 5개 5개 총 10개가 되게 만들어주는게 목표다.

# 이게 bi_lstm class에 구현하였다.

[[[-0.16927719 -0.03208148 -0.06807908  0.06336762 -0.10320626]
  [-0.29778847 -0.08916474 -0.13179946  0.04271437  0.01902312]]

 [[-0.2558721  -0.20258126 -0.03273657 -0.03389228 -0.04165369]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.23025228 -0.14986126 -0.0051727  -0.02103105 -0.00649853]
  [-0.36585653 -0.2178367  -0.02895238 -0.09726984  0.05512087]]

 [[-0.03548581 -0.06285039 -0.02809303 -0.03392856  0.02587362]
  [-0.15940066 -0.18311465 -0.09469349 -0.3778743   0.20970593]]]
shape of fw :  (4, 2, 5)
[[[ 0.28038213 -0.19698335  0.14307526  0.05816071  0.08298788]
  [ 0.68603927 -0.6253854   0.11416671  0.18727653  0.02613216]]

 [[ 0.34199747 -0.42485     0.06602378  0.35614663  0.15395209]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.6339268  -0.6575606   0.03819319  0.5389036   0.12023398]
  [ 0.624697   -0.49919644  0.19411315  0.22321084  0.01862415]]

 [[ 0.8978565  -0.36872527  0.7431453   0.19124112  0.0307906 ]
 

In [0]:
final_state = tf.concat([state[0][1], state[1][1]], axis = 1)

In [23]:
with tf.Session() as sess:
  sess.run(init)
  print(sess.run(final_state))

[[-0.29778847 -0.08916474 -0.13179946  0.04271437  0.01902312  0.28038213
  -0.19698335  0.14307526  0.05816071  0.08298788]
 [-0.2558721  -0.20258126 -0.03273657 -0.03389228 -0.04165369  0.34199747
  -0.42485     0.06602378  0.35614663  0.15395209]
 [-0.36585653 -0.2178367  -0.02895238 -0.09726984  0.05512087  0.6339268
  -0.6575606   0.03819319  0.5389036   0.12023398]
 [-0.15940066 -0.18311465 -0.09469349 -0.3778743   0.20970593  0.8978565
  -0.36872527  0.7431453   0.19124112  0.0307906 ]]
